# Mnist Classification using NN

* Mnist Classification - The Hello World of deep learning ^

##### Imports

In [66]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

##### Load Mnist Dataset

In [67]:
# with_info also gives us the meta data
# as_supervised returns us the dataset in image, label pairs
mnist_data, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [68]:
mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

In [69]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\Palmg\\tensorflow_datasets\\mnist\\3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [70]:
# 10% of train data becomes validation data
validation_size = 0.1 * mnist_info.splits['train'].num_examples
validation_size = tf.cast(validation_size, tf.int64)
validation_size

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [71]:
test_size = mnist_info.splits['test'].num_examples
test_size = tf.cast(test_size, tf.int64)
test_size

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

##### Scaling

In [72]:
# Function to scale all pixel values from 0-255 to 0-1
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.
    return image, label

In [73]:
# Scaling using map and previously defined function
scaled_training_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

##### Shuffling

In [74]:
buffer_size = 5000
shuffled_training_data = scaled_training_data.shuffle(buffer_size)

##### Preprocessed data ready

In [75]:
# Take the first 6000 as validation data and the rest 54000 as train_data
validation_data = shuffled_training_data.take(validation_size)
train_data = shuffled_training_data.skip(validation_size)

##### Batching

In [76]:
# Batch size
batch_size = 64

In [77]:
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(validation_size)
test_data = scaled_test_data.batch(test_size)

In [78]:
# Since there's only 1 batch we can use next() to load the whole batch at once
validation_inputs, validation_targets = next(iter(validation_data))

# Model

##### Hyperparameters

In [79]:
# 28 x 28 = 784
input_size = 784
# 0-9 -> 10 outputs
output_size = 10
# 50 neural units per hidden layer
hidden_layer_size = 100

##### Model architecture

In [80]:
model = tf.keras.Sequential([
    # Flattens a tensor into a vector
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    # First hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    # Second hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    # Output layer
    tf.keras.layers.Dense(output_size, activation='softmax')
])

##### Optimizer and Loss Function

In [81]:
# adam optimizer mvp
# sparse_categorical_crossentropy applies one-hot encoding for us.
# If we had one-hot encoding then choose categorical_crossentropy
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

##### Training

In [82]:
epochs = 5

model.fit(train_data,
          epochs = epochs,
          validation_data=(validation_inputs, validation_targets),
          verbose = 1)

Epoch 1/5


844/844 [==============================] - 2s 1ms/step - loss: 0.2947 - accuracy: 0.9142 - val_loss: 0.1775 - val_accuracy: 0.9452
Epoch 2/5
844/844 [==============================] - 1s 1ms/step - loss: 0.1210 - accuracy: 0.9641 - val_loss: 0.1134 - val_accuracy: 0.9652
Epoch 3/5
844/844 [==============================] - 1s 1ms/step - loss: 0.0853 - accuracy: 0.9740 - val_loss: 0.1013 - val_accuracy: 0.9687
Epoch 4/5
844/844 [==============================] - 1s 1ms/step - loss: 0.0677 - accuracy: 0.9788 - val_loss: 0.0886 - val_accuracy: 0.9718
Epoch 5/5
844/844 [==============================] - 1s 1ms/step - loss: 0.0535 - accuracy: 0.9830 - val_loss: 0.0678 - val_accuracy: 0.9788
